In [8]:
import json
import requests
import sys
from datetime import datetime, time, timedelta
from selenium import webdriver
GECKO_PATH = 'C:\\Users\\25052288840\\Downloads\\chromedriver.exe'
driver = webdriver.Chrome(GECKO_PATH)
sys.path.insert(0, '../../../commons/')
sys.path.insert(0, '../../../virasana/')
from virasana.integracao.due import raspa_due

VIRASANA_URL = "https://ajna.labin.rf08.srf/virasana/"

In [9]:
diaapesquisar = datetime.today() - timedelta(days=4)
datainicial = datetime.strftime(datetime.combine(diaapesquisar, time.min), '%Y-%m-%d  %H:%M:%S')
datafinal = datetime.strftime(datetime.combine(diaapesquisar, time.max), '%Y-%m-%d %H:%M:%S')
print(datainicial, datafinal)

2019-11-06  00:00:00 2019-11-06 23:59:59


In [11]:
def do_update(tipo_manifesto):
    conteineres_ids = raspa_containers_sem_due(
        datainicial, datafinal, tipo_manifesto)
    print(len(conteineres_ids))
    try:
        raspa_due.auth_suite_rfb(driver)
        %time conteineres_listadue = raspa_due.get_dues_pos_acd(driver, conteineres)
        # print(conteineres_listadue)
        %time due_detalhe = raspa_due.detalha_dues(driver, conteineres_listadue)
        # print(conteineres_due)
    finally:
        driver.close()
    print(len(conteineres_listadue))
    print(len(due_detalhe))
    pacote_carregamento = {}
    for conteiner, numeros_dues in conteineres_listadue.items():
        _id = conteineres_ids[conteiner]
        lista_dues = []
        for due in numeros_dues:
            if due is not None:
                pacote = raspa_due.monta_due_ajna(due_detalhe[due])
                lista_dues.append({'numero': due, **pacote})
        if numeros_dues and len(numeros_dues) > 0:
            pacote_carregamento[_id] = lista_dues
    r = requests.post(VIRASANA_URL + "dues/update", json=pacote_carregamento)
    print(r.status_code)
    print(r.text)

NameError: name 'tipo_manifesto' is not defined

In [ ]:
for tipo_manifesto in (None, 'lce'):
    do_update(tipo_manifesto)